# Transfer learning using Sentence Transformers and Scikit-Learn

In this example, we'll be demonstrating how to simply implement transfer learning using SuperDuperDB.
You'll find related examples on vector-search and simple training examples using scikit-learn in the 
the notebooks directory of the project. Transfer learning leverages similar components, and may be used synergistically with vector-search. Vectors are, after all, simultaneously featurizations of 
data and may be used in downstream learning tasks.

Let's first connect to MongoDB via SuperDuperDB, you read explanations of how to do this in 
the docs, and in the `notebooks/` directory.

In [2]:
from superduperdb import superduper
from superduperdb.db.mongodb.query import Collection
import pymongo

db = superduper(
    pymongo.MongoClient().documents
)

collection = Collection('transfer')

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


We'll use textual data labelled with sentiment, to test the functionality. Transfer learning 
can be used on any data which can be processed with SuperDuperDB models.

In [3]:
import numpy
from datasets import load_dataset

from superduperdb.container.document import Document as D

data = load_dataset("imdb")

train_data = [
    D({'_fold': 'train', **data['train'][int(i)]}) 
    for i in numpy.random.permutation(len(data['train']))
][:10000]

valid_data = [
    D({'_fold': 'valid', **data['test'][int(i)]}) 
    for i in numpy.random.permutation(len(data['test']))
][:1000]

db.execute(collection.insert_many(train_data))

r = db.execute(collection.find_one())
r

  0%|          | 0/3 [00:00<?, ?it/s]

INFO:root:found 0 uris


Document({'_id': ObjectId('64c8f2fc13655f5a0a9cb0f9'), '_fold': 'train', 'text': "A bunch of American students and their tutor decide to visit the ugliest part of Ireland in order to study ancient religious practices. Despite being repeatedly warned about the dangers of straying off the beaten path (by the local creepy Irish guy, natch), they do just that, and wind up with their insides on the outside courtesy of a family of inbred cannibals (the descendants of the infamous Sawney Bean clan, who according to the film's silly plot, upped sticks from Scotland and settled on the Emerald Isle).<br /><br />If you think that porn stars plus low budget horror automatically equals tons of nudity and terrible acting, then think again: Evil Breed is bristling with adult stars, but in fact, there's not nearly as much nudity as one might expect given the 'talent' involved, and the acting, although far from Oscar worthy, ain't all that bad (with the exception of Ginger Lynn Allen, who we know can d

Let's create a SuperDuperDB model based on a `sentence_transformers` model.
You'll notice that we don't necessarily need a native SuperDuperDB integration to a model library 
in order to leverage its power with SuperDuperDB. For example, in this case, we just need 
to configure the `Model` wrapper to interoperate correctly with the `SentenceTransformer` class. After doing this, we can link the model to a collection, and daemonize the model using the `watch=True` keyword:

In [6]:
from superduperdb.container.model import Model
import sentence_transformers

from superduperdb.ext.numpy.array import array

m = Model(
    identifier='all-MiniLM-L6-v2',
    object=sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2'),
    encoder=array('float32', shape=(384,)),
    predict_method='encode',
    batch_predict=True,
)

m.predict(
    X='text',
    db=db,
    select=collection.find(),
    watch=True
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/y9/b74b9yj906s_wtj0rrh2lf7c0000gn/T/tmp9qh0wm8h
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/y9/b74b9yj906s_wtj0rrh2lf7c0000gn/T/tmp9qh0wm8h/_remote_module_non_scriptable.py
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Now that we've created and added the model which computes features for the `"text"`, we can train a 
downstream model using Scikit-Learn:

In [7]:
from sklearn.svm import SVC

model = superduper(
    SVC(gamma='scale', class_weight='balanced', C=100, verbose=True),
    postprocess=lambda x: int(x)
)

model.fit(
    X='text',
    y='label',
    db=db,
    select=collection.find().featurize({'text': 'all-MiniLM-L6-v2'}),
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 58573.37it/s]


[LibSVM]...................*.........*
optimization finished, #iter = 28120
obj = -7557.408109, rho = 0.295979
nSV = 6656, nBSV = 0
Total nSV = 6656


SVC(C=100, class_weight='balanced', verbose=True)

Now that the model has been trained, we can apply the model to the database, also daemonizing the model 
with `watch=True`.

In [8]:
model.predict(
    X='text',
    db=db,
    select=collection.find().featurize({'text': 'all-MiniLM-L6-v2'}),
    watch=True,
)

To verify that this process has worked, we can sample a few records, to inspect the sanity of the predictions

In [14]:
r = next(db.execute(collection.aggregate([{'$sample': {'size': 1}}])))
print(r['text'][:100])
print(r['_outputs']['text']['svc'])

The writers probably had no experience in the army, and probably never glanced at a history book, bu
0
